In [3]:
import numpy as np
import matplotlib.pyplot as plt
from database_methods import Table, to_show, head
import sqlite3
con = sqlite3.connect('database_results.db')
cursor = con.cursor()

In [4]:
mas = Table(connection=con, name='masters')
exp = Table(con, name = 'experiments')
meas = Table(con, name = 'measurements')

In [19]:
print(mas.columns)
print(exp.columns)
print(meas.columns)
print('-'*80)
mas.show_all()
exp.show_all()

CATEGORY, PERIOD, AMPLITUDE, MASTER, FWHM, R, S
DATE, MASTER_ID, DIP, FOLDER
EXPERIMENT_ID, NP, WAVELENGTH, FILE, AR_TH, AR_DIP, AR_NOISE, EXC_THS, RATE, INFO, COUNT, X, Y, CON, INP, INPPX, IBGPX, STD, stdX, stdY, stdCON, stdINP, stdINPPX, stdIBGPX, stdSTD, NOTES, AR_MIN, AR_COND, EXP_TYPE
--------------------------------------------------------------------------------
ID	CATEGORY	PERIOD	AMPLITUDE	MASTER	FWHM	R	S
1	600	405	45	M5	23.0	8.0	15.9
2	600	405	53	P3	25.0	19.7	3.8
3	650	452	32	K4	12.5	11.9	17.8
4	650	452	40	K5	15.0	18.0	8.8
5	750	528	24	L3	7.0	13.8	78.5
6	750	526	30	Q4	9.5	22.0	30.7
ID	DATE	MASTER_ID	DIP	FOLDER
1	20-02-25	2	615	20_02_25_P3/
2	20-02-26	6	727	20_02_26_Q3/
3	20-03-16	4	658	20_03_16_K5/
5	20-02-25	1	598	20_02_25_M5/
6	20-03-16	3	647	20_03_16_K4/
7	20-04-15	5	740	20_04_15_L3/
8	20-04-14	1	606	20_04_14_M5/


In [18]:
con.commit()

In [34]:
# 
columns = 'MASTER,  AVG(DIP), AVG(INP), AVG(INP/IBGPX), AVG(CON), AVG(INPPX), AVG(IBGPX)'
data = con.execute("""
SELECT {}
FROM 'masters' as MAS 
INNER JOIN 'experiments' as EXP 
ON MAS.ID = EXP.MASTER_ID
INNER JOIN 'measurements' as MEAS
ON EXP.ID = MEAS.EXPERIMENT_ID
GROUP BY MASTER
ORDER BY DIP DESC
;
   """.format(columns))

head(columns)

for row in data:
    print(str(to_show(row)).replace(', ', '\t').replace("'", "")[1:-1])


MASTER	 AVG(DI	AVG(INP	AVG(INP	AVG(CON	AVG(INP	AVG(IBG
L3	740.00	18.53	54.94	3.31	1.15	0.3505
Q4	727.00	25.81	58.02	3.81	1.73	0.4763
K5	658.00	14.79	48.99	5.44	1.64	0.3078
K4	647.00	6.18	24.67	3.32	0.9056	0.2691
P3	615.00	5.25	20.50	7.27	1.81	0.2621
M5	606.00	2.54	11.68	6.60	1.42	0.2196


In [49]:
# 
columns = ' MASTER, DATE, FILE, WAVELENGTH, NP, DIP, COUNT, RATE, CON, COUNT, X*Y, INP, INPPX, IBGPX, STD, EXP_TYPE,  NOTES'
data = con.execute("""
SELECT {}
FROM 'masters' as MAS 
INNER JOIN 'experiments' as EXP 
ON MAS.ID = EXP.MASTER_ID
INNER JOIN 'measurements' as MEAS
ON EXP.ID = MEAS.EXPERIMENT_ID
WHERE category = 750

ORDER BY date DESC
;
   """.format(columns))

head(columns)

i=0
for row in data:
    print(str(to_show(row)).replace(', ', '\t').replace("'", "")[1:-1])
    i+=1
    
    
print('---\n{}'.format(i))

 MASTER	DATE	FILE	WAVELEN	NP	DIP	COUNT	RATE	CON	COUNT	X*Y	INP	INPPX	IBGPX	STD	EXP_TYP	 NOTES
Q4	20-04-2	raw_02_	732	80	729	138	47.93	11.33	138	225.36	25.61	3.24	0.2904	0.4365	-	-
Q4	20-04-2	raw_03_	726	80	729	158	45.75	2.88	158	675.36	31.46	0.8306	0.2879	0.3733	-	-
Q4	20-04-2	raw_25_	732	60	729	201	71.88	6.06	201	277.04	20.16	1.55	0.2579	0.3590	-	-
Q4	20-04-2	raw_17_	732	80	729	226	61.15	6.50	226	306.35	25.23	1.76	0.2741	0.4109	strep	-
L3	20-04-1	raw_05_	743	80	740	64	19.23	3.29	64	535.99	27.79	1.14	0.3563	0.4479	-	-
L3	20-04-1	raw_04_	742	80	740	56	21.43	2.47	56	534.84	16.30	0.6762	0.2772	0.3531	-	-
L3	20-04-1	raw_06_	744	80	740	39	24.50	2.89	39	585.91	27.43	1.12	0.3970	0.5096	-	bubles
L3	20-04-1	raw_07_	745	80	740	94	29.50	3.66	94	472.75	33.34	1.52	0.4178	0.5305	-	nice
L3	20-04-1	raw_08_	746	80	740	86	24.14	3.36	86	458.63	31.20	1.45	0.4340	0.5498	-	nice
L3	20-04-1	raw_09_	748	80	740	67	24.06	3.91	67	305.28	22.18	1.77	0.4542	0.5750	-	-
L3	20-04-1	raw_10_	750	80	740	29	29.17	3.09	29	22

In [50]:
columns = 'NP, WAVELENGTH, AR_TH, AR_NOISE, EXC_THS'
data = con.execute("""
SELECT {} FROM 'measurements'
WHERE WAVELENGTH=745
ORDER BY WAVELENGTH ASC;
   """.format(columns))

print(columns.replace(', ','\t'))
for row in data:
    print(str(row).replace(', ', '\t').replace("'", "")[1:-1])

NP	WAVELENGTH	AR_TH	AR_NOISE	EXC_THS
80	745	39.0	0.001	[2	17.580000000000002]
60	745	30.0	0.001	[2	17.580000000000002]
100	745	38.0	0.001	[2.5	17.580000000000002]
100	745	100.0	0.001	None


In [213]:
con.execute("""
UPDATE 'measurements' SET NOTES = 'weak' WHERE COUNT = 165
""")

In [215]:
meas.commit()

In [9]:
con.close()